In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
# import re
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# List of URLs to scrape
urls = [
    "https://jiji.ng/agriculture-and-foodstuff?page=1",
    "https://jiji.ng/agriculture-and-foodstuff?page=2",
    "https://jiji.ng/agriculture-and-foodstuff?page=3",
    "https://jiji.ng/agriculture-and-foodstuff?page=4",
    "https://jiji.ng/agriculture-and-foodstuff?page=5",
    "https://jiji.ng/agriculture-and-foodstuff?page=6",
    "https://jiji.ng/agriculture-and-foodstuff?page=7",
    "https://jiji.ng/agriculture-and-foodstuff?page=8",
    "https://jiji.ng/agriculture-and-foodstuff?page=9",
]

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/115.0 Safari/537.36"
    )
}

# Configure retries for requests
session = requests.Session()
retry = Retry(
    total=5,                 # retry up to 5 times
    backoff_factor=1,        # wait 1s, then 2s, then 4s, ...
    status_forcelist=[500, 502, 503, 504]
)
adapter = HTTPAdapter(max_retries=retry)
session.mount("http://", adapter)
session.mount("https://", adapter)

In [3]:
# List of URLs to scrape
urls = [
    "https://jiji.ng/agriculture-and-foodstuff?page=1",
    "https://jiji.ng/agriculture-and-foodstuff?page=2",
    "https://jiji.ng/agriculture-and-foodstuff?page=3",
    "https://jiji.ng/agriculture-and-foodstuff?page=4",
    "https://jiji.ng/agriculture-and-foodstuff?page=5",
    "https://jiji.ng/agriculture-and-foodstuff?page=6",
    "https://jiji.ng/agriculture-and-foodstuff?page=7",
    "https://jiji.ng/agriculture-and-foodstuff?page=8",
    "https://jiji.ng/agriculture-and-foodstuff?page=9",
]

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/115.0 Safari/537.36"
    )
}

# Configure retries
session = requests.Session()
retry = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[500, 502, 503, 504]
)
adapter = HTTPAdapter(max_retries=retry)
session.mount("http://", adapter)
session.mount("https://", adapter)

# Container for product descriptions
product_descriptions = []

# Loop through all the URLs
for url in urls:
    try:
        response = session.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except Exception as e:
        print(f"Skipping {url}: {e}")
        continue

    soup = BeautifulSoup(response.text, "html.parser")

    # Find all description containers
    containers = soup.find_all("div", class_="b-list-advert-base__description-text")

    if not containers:
        print(f"No descriptions found on {url}")

    for container in containers:
        desc_text = container.get_text(strip=True)
        if desc_text:
            product_descriptions.append(desc_text)

# Save to DataFrame
df = pd.DataFrame(product_descriptions, columns=["Description"])

print(df.head(10))
print("Total descriptions extracted:", len(df))

                                         Description
0  We sell the best made in Nigeria battery cages...
1  Weaners, grower and Adult Rabbits available fo...
2  Industrial pelleting machine \nUsed in product...
3  This greenhouse is cover farm against sunlight...
4            For safe drinking water to farm animals
5  KTC Blender Olive Oil and Sunflower Oil good f...
6  This flower sack is used for bagging and repac...
7  Heavy duty rice milling machine with elevator ...
8                             Animal feed pelletizer
9     These incubators uses both AC and DC circuits.
Total descriptions extracted: 216


In [4]:
product_titles = []  

# Loop through all the urls
for url in urls:
    try:
        response = session.get(url, headers=headers, timeout=7)  
        response.raise_for_status()
    except Exception as e:
        print(f"Skipping {url}: {e}")
        continue  # skip failed pages

    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find all product title containers
    containers = soup.find_all("div", class_="b-advert-title-inner qa-advert-title b-advert-title-inner--div")
    
    if not containers:
        print(f"No product title found on {url}")  
    
    for container in containers:
        title_text = container.get_text(strip=True)
        # Clean up text
        product = title_text.replace("\n", " ").strip()
        if product:
            product_titles.append(product)

print("Extracted titles:", product_titles[:10])  
print("Number of titles:", len(product_titles))



Extracted titles: ['Made in Nigeria Battery Cage for Layers', 'Healthy 1-Year-Old Noilers for Sale – Males Females Available', 'Industrial Pellet Mill', 'Greenhouse Cover 10m*30m/1000sqm', 'Pig Nipple Drinkers', 'KTC Blended Olive Oil and Sunflower Oil 5litres ×4', '50kg Flower Sack', '5in1 Rice Milling Machine With Diesel Engine', 'Pellet Machine', 'Solar Egg Incubator 56 Eggs Capacity']
Number of titles: 216


In [5]:
location_list = []   

# Loop through all the urls
for url in urls:
    try:
        response = session.get(url, headers=headers, timeout=2)  
        response.raise_for_status()
    except Exception as e:
        print(f"Skipping {url} due to error: {e}")
        continue  # skip failed pages

    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find all location containers
    containers = soup.find_all("span", class_="b-list-advert__region__text")
    
    if not containers:
        print(f"No product location found on {url}")  # clearer message
    
    for container in containers:
        title_text = container.get_text(strip=True)
        location = title_text.replace(".", "").replace("\n", ",").strip()
        if location:
            location_list.append(location)

print("Extracted locations:", location_list[:10])  
print("Total locations extracted:", len(location_list))


Extracted locations: ['Delta, Warri', 'Lagos, Ikeja', 'Lagos, Ojo', 'Lagos, Lagos Island (Eko)', 'Ogun, Obafemi-Owode', 'Lagos, Lagos Island (Eko)', 'Lagos, Lagos Island (Eko)', 'Lagos, Ojo', 'Ogun, Obafemi-Owode', 'Oyo, Ibadan']
Total locations extracted: 216


In [6]:
price_texts = []  
prices = []       

# Loop through all the urls
for url in urls:
    try:
        response = session.get(url, headers=headers, timeout=2)  # use session.get for retries
        response.raise_for_status()
    except Exception as e:
        print(f"Skipping {url}: {e}")
        continue  # skip failed pages

    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find all price containers
    containers = soup.find_all("div", class_="qa-advert-price")
    
    if not containers:
        print(f"No product price found on {url}")  # debug
    
    for container in containers:
        raw_price = container.get_text(strip=True)
        
        # Clean price text
        clean_price = (
            raw_price.replace("\n", " ")
                     .replace("₦", "")
                     .replace(",", "")
                     .strip()
        )

        price_texts.append(clean_price)

        # Extract only digits (no regex)
        numeric = "".join(ch for ch in clean_price if ch.isdigit())
        if numeric:
            prices.append(int(numeric))
        else:
            prices.append(None)  # keep alignment for missing prices

# Show results
print("Extracted price texts (sample):", price_texts[:20])  
print("Number of price texts:", len(price_texts))
print("Number of numeric prices:", len(prices))


Extracted price texts (sample): ['250000', '6000', '890000', '1500', '5500', '93000', '300', '9000000', '850000', '130000', '560000', '120000', '75000', '75000', '30000', '280000', '8000', '60000', '500000', '65000']
Number of price texts: 216
Number of numeric prices: 216


In [7]:
data = {
    "Title": product_titles,
    "Description": product_descriptions,
    "Location": location_list,
    "Price": prices
}

df = pd.DataFrame(data)
df.index = df.index + 1
df

,Title,Description,Location,Price
1,Made in Nigeria Battery Cage for Layers,We sell the best made in Nigeria battery cages...,"Delta, Warri",250000
2,Healthy 1-Year-Old Noilers for Sale – Males Fe...,"Weaners, grower and Adult Rabbits available fo...","Lagos, Ikeja",6000
3,Industrial Pellet Mill,Industrial pelleting machine \nUsed in product...,"Lagos, Ojo",890000
4,Greenhouse Cover 10m*30m/1000sqm,This greenhouse is cover farm against sunlight...,"Lagos, Lagos Island (Eko)",1500
5,Pig Nipple Drinkers,For safe drinking water to farm animals,"Ogun, Obafemi-Owode",5500
...,...,...,...,...
212,Stainless Fish Dryer for Farmer,Stainless steel fish dryer. Durable for drying...,"Lagos, Ikeja",1450000
213,Pig Nipple Drinker,"A simple structure composed of shell, mandrill...","Lagos, Orile",10500
214,Trimmer Line 2.5mm/256m,It gives you a neat work,"Lagos, Ojo",10000
215,Complete (Set of 6) Garri Machines,This complete garri processing machine comes w...,"Osun, Osogbo",5500000


In [8]:
df.to_csv("Jiji-agriculture-foodstuff.csv", index=False)

In [ ]:
import os
print(os.getcwd())

C:\Users\user
